# Time evolution with PennyLane

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Gopal-Dahale/sqte/blob/main/1_tfim_time_evolve.ipynb)

## Setup

In [ ]:
# !pip install -q pennylane

In [1]:
import pennylane as qml
import numpy as np
from pennylane.devices.qubit import create_initial_state, apply_operation
from time import time
from multiprocessing import Pool
from ipywidgets import interact, FloatSlider
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import scipy

In [2]:
def tfim_hamiltonian(n, J = 1.0, h = 1.0):
    coeffs = []
    ops = []
    
    # ZZ interaction terms
    for i in range(n - 1):
        coeffs.append(J)
        ops.append(qml.PauliZ(i) @ qml.PauliZ(i + 1))
    
    # Transverse field terms (X terms)
    for i in range(n):
        coeffs.append(h)
        ops.append(qml.PauliX(i))
    
    return qml.Hamiltonian(coeffs, ops)

H = tfim_hamiltonian(4, J=1.0, h=0.5)

In [3]:
def insert_zero_sorted(arr):
    if 0 in arr:
        return arr  # Zero is already present
    
    # Find the insertion index for zero
    idx = np.searchsorted(arr, 0)  # Finds the index where 0 should be inserted
    return np.insert(arr, idx, 0)

In [4]:
J_list = insert_zero_sorted(np.linspace(-2, 2, 20))
h_list = insert_zero_sorted(np.linspace(0, 4, 20))
t_list = np.linspace(0, np.pi, 3)

n_qubits = 4
state = create_initial_state(range(n_qubits))

# Function to generate Hamiltonian for given (J, h)
def generate_hamiltonian(params):
    J, h = params
    return (J, h, tfim_hamiltonian(n_qubits, J, h))

# Parallelize Hamiltonian creation
start = time()
H_list = [generate_hamiltonian((J, h)) for J in J_list for h in h_list]

# Convert list to dictionary for fast lookup
H_dict = {(J, h): H for J, h, H in H_list}
del H_list
print(f"Hamiltonian precompute time: {time() - start:.4f} sec")

Hamiltonian precompute time: 0.2347 sec


In [5]:
# Function to evolve state for given (t, J, h)
def evolve_state(params):
    t, J, h, H = params
    EvoH = qml.evolve(H, coeff=1)
    return apply_operation(EvoH, state).flatten()
    
# Parallelize time evolution
start = time()
with Pool() as pool:
    results = pool.map(evolve_state, [(t, J, h, H_dict[(J, h)]) for t in t_list for J in J_list for h in h_list])

results = np.array(results).reshape(len(t_list), len(J_list), len(h_list), 2**n_qubits)
print(f"Time evolution duration: {time() - start:.4f} sec")

Time evolution duration: 6.8207 sec


In [6]:
results.shape

(3, 21, 20, 16)

In [7]:
# Define computational basis states
x_vals = np.arange(2**n_qubits) 

# Interactive plot function
def plot_probabilities(t_idx, J_idx, h_idx):
    clear_output(wait=True)  # Speed up rendering by clearing old plots
    
    psi = results[t_idx, J_idx, h_idx]  # Extract state vector
    probabilities = np.abs(psi) ** 2  # Compute probabilities

    # Plot
    plt.figure(figsize=(8, 5))
    plt.bar(x_vals, probabilities, color='blue', alpha=0.7)
    plt.xlabel("Computational Basis State (x)")
    plt.ylabel("Probability |ψ(x, t)|²")
    plt.title(f"TFIM Evolution: $t={t_list[t_idx]:.2f}, J={J_list[J_idx]:.2f}, h={h_list[h_idx]:.2f}$")
    plt.ylim(0, 1)  # Probability range
    plt.xticks(x_vals)  # Show x values
    plt.show()

In [8]:
# Create sliders with actual values 
t_slider = widgets.SelectionSlider(options=[(f"{t:.2f}", i) for i, t in enumerate(t_list)], description="t")
J_slider = widgets.SelectionSlider(options=[(f"{J:.2f}", i) for i, J in enumerate(J_list)], description="J")
h_slider = widgets.SelectionSlider(options=[(f"{h:.2f}", i) for i, h in enumerate(h_list)], description="h")

# Create interactive UI
ui = widgets.VBox([t_slider, J_slider, h_slider])
out = widgets.interactive_output(plot_probabilities, {'t_idx': t_slider, 'J_idx': J_slider, 'h_idx': h_slider})

# Display widgets
display(ui, out)

Output()